In [37]:
#!/usr/bin/python
#
# -------------------------------------------------------------------------
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from mpl_toolkits.mplot3d import Axes3D
from alborex_functions import *
import logging
from matplotlib import rcParams
rcParams.update({'font.size': 16, 'font.family': 'Times New Roman'})
plottemp, plotsalinity, plotchloro = 0, 0, 1

In [2]:
def configure_logging():
    logger = logging.getLogger("alborex_3Dscatter_logger")
    logger.setLevel(logging.DEBUG)
    # Format for our loglines
    formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    # Setup console logging
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    ch.setFormatter(formatter)
    logger.addHandler(ch)
    # Setup file logging as well
    fh = logging.FileHandler('/home/ctroupin/logs/alborex_3Dscatter_plot.log')
    fh.setLevel(logging.DEBUG)
    fh.setFormatter(formatter)
    logger.addHandler(fh)
    return logger
logger = configure_logging()

In [26]:
# Plot configuration
dlon, dlat = 1., 1.
coordinates = np.array((-0.775, -0.3, 36.71, 37.19))
depthmin = 0.0
depthmax = 300.0

# Glider files: can be local or through the thredds server
gliderfile1 = 'dep0012_ideep00_ime-sldeep000_L1_2014-05-25_data_dt.nc'
gliderfile2 = 'dep0005_icoast00_ime-slcost000_L1_2014-05-25_data_dt.nc'

if 'gher' in os.uname()[1]:
    gliderdir = '/home/ctroupin/DataOceano/Alborex/Glider'
    gliderfile1 = os.path.join(gliderdir, gliderfile1)
    gliderfile2 = os.path.join(gliderdir, gliderfile2)
    figdir = '/home/ctroupin/IMEDEA/Alborex/figures/201610/'
else:
    gliderdir = 'http://thredds.priv.socib.es/thredds/dodsC/auv/glider/'
    gliderfile1 = (gliderdir, 'ideep00-ime_sldeep000/L1/2014/dep0012_ideep00_ime-sldeep000_L1_2014-05-25_data_dt.nc')
    gliderfile2 = (gliderdir, 'icoast00-ime_slcost000/L1/2014/dep0005_icoast00_ime-slcost000_L1_2014-05-25_data_dt.nc')
    ctdfile = ('http://thredds.priv.socib.es/thredds/dodsC/research_vessel/ctd/',
           'socib_rv-scb_sbe9002/L1/2014/dep0007_socib-rv_scb-sbe9002_L1_2014-05-25.nc')
    figdir = '/home/ctroupin/public_html/Alborex/figures_20161006/'

In [5]:
logger.info('---Reading data---')

logger.info('Loading coast')
loncoast, latcoast = alborex_load_coast(coastdir, coastfile, valex)

logger.info('Loading CTD data')
# lonCTD, latCTD, depthCTD, tempCTD, chloroCTD = alborex_load_ctd(ctdfile)

logger.info('Loading coordinates and temperature data from gliders (with sub-sampling)')
NN = 5
longlider1, latglider1, depthglider1, temperatureglider1 = alborex_loadglider_subsample(gliderfile1, NN)
longlider2, latglider2, depthglider2, temperatureglider2 = alborex_loadglider_subsample(gliderfile2, NN)

logger.info('Loading chloro from glider')
chloroglider1 = alborex_loadglider_varname(gliderfile1, 'chlorophyll', NN)
chloroglider2 = alborex_loadglider_varname(gliderfile2, 'chlorophyll', NN)

logger.info('Loading salinity from glider')
salinityglider1 = alborex_loadglider_varname(gliderfile1, 'salinity_corrected_thermal', NN)
salinityglider2 = alborex_loadglider_varname(gliderfile2, 'salinity_corrected_thermal', NN)

logger.info('Remove infinite values in the coordinate arrays')
longlider1[longlider1 > coordinates[1]] = np.nan
longlider2[longlider2 > coordinates[1]] = np.nan

2016-10-11 14:15:43,434 - alborex_3Dscatter_logger - INFO - ---Reading data---
2016-10-11 14:15:43,469 - alborex_3Dscatter_logger - INFO - Loading coast
2016-10-11 14:15:43,885 - alborex_3Dscatter_logger - INFO - Loading CTD data
2016-10-11 14:15:43,886 - alborex_3Dscatter_logger - INFO - Loading coordinates and temperature data from gliders (with sub-sampling)
2016-10-11 14:15:50,101 - alborex_3Dscatter_logger - INFO - Loading chloro from glider
2016-10-11 14:15:50,201 - alborex_3Dscatter_logger - INFO - Loading salinity from glider
2016-10-11 14:15:50,301 - alborex_3Dscatter_logger - INFO - Remove infinite values in the coordinate arrays


# Make the plots

In [6]:
# Parameters for the plot
props = dict(boxstyle='round', facecolor='white', alpha=0.99)
angles = np.array((55, -40))

## Function go get the 3D scatter plots

In [31]:
def alborex_scatter_3D(lonlist, latlist, depthlist, fieldlist, 
                       varunits, varmin, varmax, dvar, cmap, 
                       coordinates, depths, 
                       figdir, figname): 
    
    bounds = np.arange(varmin, varmax + .0001, dvar)
    
    if varmin == 0.01:
        norm = colors.LogNorm(vmin=varmin, vmax=varmax)
    else:
        norm = colors.Normalize(vmin=varmin, vmax=varmax)

    fig = plt.figure(num=None, figsize=(14, 8))
    ax = fig.gca(projection='3d')
    fig.patch.set_facecolor('white')

    # Plot the glider profiles
    for lon, lat, field, depth in zip(lonlist, latlist, fieldlist, depthlist):
        scat3D = ax.scatter(lon, lat, -depth, s=5, c=field, edgecolor='None', cmap=cmap,
                            norm=norm, zorder=4)

    # Add the colorbar
    cbar = fig.colorbar(scat3D, cmap=cmap, orientation='vertical', pad=0.05, aspect=15, 
                        shrink=0.8, extend='both')
    cbar.set_label(varunits, fontname='Times New Roman', fontsize=18)
    cbar.set_clim(varmin, varmax)
    
    if varmin != 0.01:
        cbar.set_ticks(bounds)
        cbar.ax.set_yticklabels(bounds, fontname='Times New Roman', fontsize=16)

    # Plot coastline
    # ax.plot3D(loncoast,latcoast,0,color='k',lw=0.5,zorder=6)
    plt.xlabel('\nLongitude')
    plt.ylabel('\nLatitude')

    # Change wall properties
    change_wall_prop(ax, coordinates, depths, angles)

    plt.savefig(os.path.join(figdir, figname), dpi=300, facecolor='w', edgecolor='w', transparent=False,
                bbox_inches='tight', pad_inches=0.1)
    # plt.show()
    plt.close()

## Some parameters for the plots

In [20]:
valex = 999

tempmin, tempmax, deltatemp = 13., 19., 1.0
cmaptemp = plt.cm.spectral

psalmin, psalmax, deltapsal = 36.5, 38.5, 0.5
cmapCTDpsal = plt.cm.RdYlBu_r

chloromin, chloromax, deltachloro = 0.01, 1.5, 0.3
cmapchloro = plt.cm.YlGnBu_r

## Plots for the deep glider

In [21]:
coordinates = np.array((-0.775, -0.3, 36.71, 37.19))
depthmin = -500.0
depthmax = 0.0
deltadepth = 100.0
depths = np.array((depthmin, depthmax, deltadepth))

logger.info('-- Generating the plots for the deep glider')

if plottemp:
    logger.info('-- Working on temperature')
    alborex_scatter_3D(longlider1, latglider1, depthglider1, temperatureglider1, '$^{\circ}$C', 
                       tempmin, tempmax, deltatemp, cmaptemp, 
                       coordinates, depths, 
                       figdir, 'Alborex_scatter3D_temperature_ideep')

if plotsalinity:
    logger.info('-- Working on salinity')
    alborex_scatter_3D(longlider1, latglider1, depthglider1, salinityglider1, '', 
                       psalmin, psalmax, deltapsal, cmappsal, 
                       coordinates, depths, 
                       figdir, 'Alborex_scatter3D_salinity_ideep')

if plotchloro:
    logger.info('-- Working on chlorophyll concentration')
    alborex_scatter_3D(longlider1, latglider1, depthglider1, chloroglider1, 'mg/m$^{3}$', 
                       chloromin, chloromax, deltachloro, cmapchloro, 
                       coordinates, depths, 
                       figdir, 'Alborex_scatter3D_chlorophyll_ideep')

2016-10-11 14:34:45,770 - alborex_3Dscatter_logger - INFO - -- Generating the plots for the deep glider
2016-10-11 14:34:45,773 - alborex_3Dscatter_logger - INFO - -- Working on temperature
2016-10-11 14:34:58,243 - alborex_3Dscatter_logger - INFO - -- Working on salinity
2016-10-11 14:35:09,838 - alborex_3Dscatter_logger - INFO - -- Working on chlorophyll concentration


## Plots for the coastal glider

In [25]:
coordinates = np.array((-0.82, -0.3, 36.76, 37.15))
depthmin = -200.0
depthmax = 0.0
deltadepth = 50.0
depths = np.array((depthmin, depthmax, deltadepth))

logger.info('-- Generating the plots for the coastal glider')

if plottemp:
    logger.info('-- Working on temperature')
    alborex_scatter_3D(longlider2, latglider2, depthglider2, temperatureglider2, '$^{\circ}$C', 
                       tempmin, tempmax, deltatemp, cmaptemp, 
                       coordinates, depths, 
                       figdir, 'Alborex_scatter3D_temperature_icoast')
    
if plotsalinity:
    logger.info('-- Working on salinity')
    alborex_scatter_3D(longlider2, latglider2, depthglider2, salinityglider2, '', 
                       psalmin, psalmax, deltapsal, cmappsal, 
                       coordinates, depths, 
                       figdir, 'Alborex_scatter3D_salinity_icoast')

if plotchloro:
    logger.info('-- Working on chlorophyll concentration')
    alborex_scatter_3D(longlider2, latglider2, depthglider2, chloroglider2, 'mg/m$^{3}$', 
                       chloromin, chloromax, deltachloro, cmapchloro, 
                       coordinates, depths, 
                       figdir, 'Alborex_scatter3D_chlorophyll_icoast')

2016-10-11 14:37:47,641 - alborex_3Dscatter_logger - INFO - -- Generating the plots for the coastal glider
2016-10-11 14:37:47,643 - alborex_3Dscatter_logger - INFO - -- Working on temperature
2016-10-11 14:37:55,949 - alborex_3Dscatter_logger - INFO - -- Working on salinity
2016-10-11 14:38:04,021 - alborex_3Dscatter_logger - INFO - -- Working on chlorophyll concentration


## Both gliders at the same time

In [39]:
coordinates = np.array((-0.775, -0.3, 36.71, 37.19))
depthmin = -500.0
depthmax = 0.0
deltadepth = 100.0
depths = np.array((depthmin, depthmax, deltadepth))

logger.info('-- Generating the plots for the two glider at the same time')

lonlist = [longlider1, longlider2]
latlist = [latglider1, latglider2]
depthlist = [depthglider1, depthglider2]

if plottemp:
    templist = [temperatureglider1, temperatureglider2]
    logger.info('-- Working on temperature')
    alborex_scatter_3D(lonlist, latlist, depthlist, templist, '$^{\circ}$C', 
                       tempmin, tempmax, deltatemp, cmaptemp, 
                       coordinates, depths, 
                       figdir, 'Alborex_scatter3D_temperature_ideep_icoast')

if plotsalinity:
    psallist = [salinityglider1, salinityglider2]
    logger.info('-- Working on salinity')
    alborex_scatter_3D(lonlist, latlist, depthlist, psallist, '', 
                       psalmin, psalmax, deltapsal, cmappsal, 
                       coordinates, depths, 
                       figdir, 'Alborex_scatter3D_salinity_ideep_icoast')

if plotchloro:
    chlorolist = [chloroglider1, chloroglider2]
    logger.info('-- Working on chlorophyll concentration')
    alborex_scatter_3D(lonlist, latlist, depthlist, chlorolist, '', 
                       chloromin, chloromax, deltachloro, cmapchloro, 
                       coordinates, depths, 
                       figdir, 'Alborex_scatter3D_chloro_ideep_icoast')

2016-10-11 15:07:07,606 - alborex_3Dscatter_logger - INFO - -- Generating the plots for the two glider at the same time
2016-10-11 15:07:07,608 - alborex_3Dscatter_logger - INFO - -- Working on chlorophyll concentration
